In [1]:
import pandas as pd
import csv
from IPython.display import clear_output
from collections import Counter

In [2]:
def parse_domain(dom_str):

    def sort_domains(domain_list):
        idx = sorted(range(len(domain_list)), key=lambda k: domain_list[k][1][0])
        return [domain_list[i] for i in idx]

    unsorted = [[it[0], list(map(int, it[1].split(':')))] for it in (dom.split(';') for dom in dom_str.split('|'))]

    return sort_domains(unsorted)


def count_real_domains(dom_list):
    return len([dom for dom in dom_list if dom[0][0] not in ['m', 'd']])


def train_seq_redundant(dom_list, test_doms):
    return set([dom[0] for dom in dom_list]).isdisjoint(test_doms)

In [7]:
all_doms = []
with open('non_test_nr.csv') as f:
    for line in f:
        all_doms += [dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d' and dom[0][0]!='m']
all_doms_counter = Counter(all_doms)
print(len(all_doms_counter))
most_common_doms = all_doms_counter.most_common(2000)
most_common_doms = [dom for dom, _ in most_common_doms]

16916


In [8]:
with open('non_test_nr.csv') as f:
    with open('non_valid.csv', 'w') as fNonValid:
        with open('non_test_nr_no_common.csv', 'w') as fNoCom:
            for line in f:
                if set([dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d']).isdisjoint(most_common_doms):
                    fNoCom.write(line)
                else:
                    fNonValid.write(line)

In [9]:
with open('non_test_nr_no_common.csv') as f:
    with open('non_test_nr_sing_no_idr.csv', 'w') as fSingND:
        with open('non_test_nr_multi_no_idr.csv', 'w') as fMultiND:
            with open('non_test_nr_sing_idr.csv', 'w') as fSingID:
                with open('non_test_nr_multi_idr.csv', 'w') as fMultiID:
                    for idx, line in enumerate(f):
                        doms = [dom[0] for dom in parse_domain(line.split(',')[1])]
                        if 'd' not in doms:
                            if count_real_domains(parse_domain(line.split(',')[1])) == 1:
                                fSingND.write(line)
                            else:
                                fMultiND.write(line)
                        else:
                            if count_real_domains(parse_domain(line.split(',')[1])) == 1:
                                fSingID.write(line)
                            else:
                                fMultiID.write(line)
                        if idx%1000000==0:
                            clear_output()
                            print(f'Processing Line {idx}')

Processing Line 4000000


In [10]:
with open('non_test_nr_sing_no_idr.csv') as f:
    sing_no_idr_counter = 0
    for line in f:
        sing_no_idr_counter += 1
print(sing_no_idr_counter)
with open('non_test_nr_multi_no_idr.csv') as f:
    mult_no_idr_counter = 0
    for line in f:
        mult_no_idr_counter += 1
print(mult_no_idr_counter)
with open('non_test_nr_sing_idr.csv') as f:
    sing_idr_counter = 0
    for line in f:
        sing_idr_counter += 1
print(sing_idr_counter)
with open('non_test_nr_multi_idr.csv') as f:
    mult_idr_counter = 0
    for line in f:
        mult_idr_counter += 1
print(mult_idr_counter)

3897594
68158
118863
5081


In [11]:
import numpy as np

sing_no_idr_ind = set(np.random.choice(a=sing_no_idr_counter, size=300, replace=False))
multi_no_idr_ind = set(np.random.choice(a=mult_no_idr_counter, size=5000, replace=False))
sing_idr_ind = set(np.random.choice(a=sing_idr_counter, size=300, replace=False))
multi_idr_ind = set(np.random.choice(a=mult_idr_counter, size=300, replace=False))

In [12]:
valid_sing_no_idr_arcs = set()
valid_multi_no_idr_arcs = set()
valid_sing_idr_arcs = set()
valid_multi_idr_arcs = set()
with open('non_test_nr_sing_no_idr.csv') as f:
    for idx, line in enumerate(f):
        if idx in sing_no_idr_ind:
            valid_sing_no_idr_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
with open('non_test_nr_multi_no_idr.csv') as f:
    for idx, line in enumerate(f):
        if idx in multi_no_idr_ind:
            valid_multi_no_idr_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
with open('non_test_nr_sing_idr.csv') as f:
    for idx, line in enumerate(f):
        if idx in sing_idr_ind:
            valid_sing_idr_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
with open('non_test_nr_multi_idr.csv') as f:
    for idx, line in enumerate(f):
        if idx in multi_idr_ind:
            valid_multi_idr_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
print(len(valid_sing_no_idr_arcs))
print(len(valid_multi_no_idr_arcs))
print(len(valid_sing_idr_arcs))
print(len(valid_multi_idr_arcs))

291
2011
262
213


In [13]:
valid_sing_no_idr_doms = set(sum([arc.split('|') for arc in valid_sing_no_idr_arcs], []))
valid_multi_no_idr_doms = set(sum([arc.split('|') for arc in valid_multi_no_idr_arcs], []))
valid_sing_idr_doms = set(sum([arc.split('|') for arc in valid_sing_idr_arcs], []))
valid_multi_idr_doms = set(sum([arc.split('|') for arc in valid_multi_idr_arcs], []))
valid_doms = valid_sing_no_idr_doms.union(valid_multi_no_idr_doms).union(valid_sing_idr_doms).union(valid_multi_idr_doms)
len(valid_doms)

2861

In [18]:
valid_arcs = set()

with open('non_test_nr_sing_no_idr.csv') as f:
    with open('valid_set_unbalanced.csv', 'w') as fValid:
        with open('non_valid.csv', 'a') as fNonValid:
            for idx, line in enumerate(f):
                if idx in sing_no_idr_ind:
                    fValid.write(line)
                    valid_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
                else:
                    fNonValid.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

with open('non_test_nr_multi_no_idr.csv') as f:
    with open('valid_set_unbalanced.csv', 'a') as fValid:
        with open('non_valid.csv', 'a') as fNonValid:
            for idx, line in enumerate(f):
                if idx in multi_no_idr_ind:
                    fValid.write(line)
                    valid_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
                else:
                    fNonValid.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

with open('non_test_nr_sing_idr.csv') as f:
    with open('valid_set_unbalanced.csv', 'a') as fValid:
        with open('non_valid.csv', 'a') as fNonValid:
            for idx, line in enumerate(f):
                if idx in sing_idr_ind:
                    fValid.write(line)
                    valid_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
                else:
                    fNonValid.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

with open('non_test_nr_multi_idr.csv') as f:
    with open('valid_set_unbalanced.csv', 'a') as fValid:
        with open('non_valid.csv', 'a') as fNonValid:
            for idx, line in enumerate(f):
                if idx in multi_idr_ind:
                    fValid.write(line)
                    valid_arcs.add('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
                else:
                    fNonValid.write(line)
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line {idx}')

Processing Line 0


In [19]:
pd.read_csv('valid_set_unbalanced.csv', header=None).sample(frac=1).reset_index(drop=True).to_csv('valid_set_unbalanced.csv', header=None, index=None)

In [20]:
from collections import Counter

valid_arc_counts = []
with open('valid_set_unbalanced.csv') as f:
    for idx, line in enumerate(f):
        valid_arc_counts.append('|'.join([dom[0] for dom in parse_domain(line.split(',')[1])]))
        if idx % 1000000 == 0:
            clear_output()
            print(f'Processing Line {idx}')

valid_arc_counts = Counter(valid_arc_counts)
print(valid_arc_counts.most_common(100))
print(len(valid_arc_counts))

Processing Line 0
[('PF16499|PF17801', 127), ('PF02563|PF10531', 53), ('PF20229|PF09828', 33), ('PF07584|PF13519', 31), ('PF06044|PF17726', 31), ('PF12633|PF01295', 30), ('PF00271|PF19833', 30), ('PF18984|PF18983', 29), ('PF16499|PF17450', 29), ('PF06218|PF06218', 28), ('PF05943|PF18945', 27), ('PF00906|PF00906', 27), ('PF17127|PF13905', 25), ('PF20066|PF19581', 23), ('PF12204|PF21053', 21), ('PF19346|PF11141', 20), ('PF06400|PF06401', 20), ('PF06268|PF06268|PF06268|PF06268', 19), ('PF03710|PF08335', 19), ('PF10568|PF17171', 18), ('PF05644|PF05644', 18), ('PF14908|PF18289', 17), ('PF17989|PF21522', 17), ('PF01074|PF09261', 16), ('PF12338|PF00101', 16), ('PF01507|PF11922', 16), ('PF00050|PF00050|PF00050', 15), ('PF13866|PF13867', 15), ('PF13657|PF07804', 15), ('PF10103|PF10103', 15), ('PF04820|PF04820', 15), ('PF01419|PF01419', 15), ('PF10703|PF17409', 15), ('PF04801|PF19725', 14), ('PF07653|PF07653', 14), ('PF04712|PF04712', 14), ('PF00088|PF00088', 14), ('PF19338|PF00243', 14), ('PF09

In [21]:
valid_arc_counter = {k:0 for k in valid_arc_counts.keys()}
with open('valid_set_unbalanced.csv') as f:
    with open('valid.csv', 'w') as fOut:
        for line in f:
            dom_arc_this_prot = '|'.join([dom[0] for dom in parse_domain(line.split(',')[1])])
            if np.random.rand() < 0.8**valid_arc_counter[dom_arc_this_prot]:
                fOut.write(line)
                valid_arc_counter[dom_arc_this_prot] += 1

In [24]:
valid_doms = set()
with open('valid.csv') as f:
    for line in f:
        prot_doms = [dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d']
        valid_doms.update(prot_doms)
print(len(valid_doms))

2860


In [27]:
with open('non_valid.csv') as f:
    with open('train_unbalanced.csv', 'w') as fOut:
        for idx, line in enumerate(f):
            if set([dom[0] for dom in parse_domain(line.split(',')[1]) if dom[0]!='d']).isdisjoint(valid_doms):
                fOut.write(line)
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Line: {idx}')

Processing Line: 25000000
